# Notebook 8: Ensemble Methods (Combinando Modelos)

## Objetivo
Aumentar a robustez e precisão combinando os resultados de múltiplos detectores de outliers.

## O Conceito
Nenhum algoritmo é perfeito. 
- O KNN é bom para anomalias globais.
- O LOF é bom para anomalias locais.
- O iForest é bom para altas dimensões.

Ao combinar seus scores (Ensemble), podemos reduzir falsos positivos e aumentar a confiança na detecção.

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.iforest import IForest
from pyod.models.combination import aom, moa, average, maximization
from pyod.utils.data import generate_data
from pyod.utils.utility import standardizer

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Gerando Dados Complexos
Vamos gerar um dataset onde um único modelo pode ter dificuldades.

In [ ]:
X_train, X_test, y_train, y_test = generate_data(n_train=500, n_test=100, n_features=3, contamination=0.05, random_state=42)

## 2. Treinando Modelos Individuais
Vamos treinar 3 modelos diferentes.

In [ ]:
# Inicializando classificadores
classifiers = {
    'KNN': KNN(n_neighbors=5),
    'LOF': LOF(n_neighbors=20),
    'Isolation Forest': IForest(n_estimators=100)
}

# Armazenando os scores
test_scores = np.zeros([X_test.shape[0], len(classifiers)])

for i, (clf_name, clf) in enumerate(classifiers.items()):
    print(f"Treinando {clf_name}...")
    clf.fit(X_train)
    # Prever e salvar scores normalizados
    test_scores[:, i] = clf.decision_function(X_test)

# É CRUCIAL padronizar os scores antes de combinar, pois
# o score do KNN (distância) tem escala diferente do iForest (caminho na árvore).
test_scores_norm = standardizer(test_scores)
print("Scores calculados e normalizados.")

## 3. Combinando Scores
O PyOD oferece métodos prontos para combinação:
1.  **Average** (Média dos scores)
2.  **Maximization** (Pega o maior score entre todos os modelos - conservador, se um modelo apitar, é anomalia).

In [ ]:
# Combinação por Média
y_by_average = average(test_scores_norm)

# Combinação por Maximização
y_by_maximization = maximization(test_scores_norm)

# Avaliando
from pyod.utils.data import evaluate_print

print("\n--- Resultados Individuais ---")
for i, clf_name in enumerate(classifiers.keys()):
    evaluate_print(clf_name, y_test, test_scores_norm[:, i])

print("\n--- Resultados Combinados ---")
evaluate_print('Combination (Average)', y_test, y_by_average)
evaluate_print('Combination (Max)', y_test, y_by_maximization)

## Conclusão
Geralmente, a combinação (especialmente Average ou AOM - Average of Maximums) tende a ser mais estável do que depender de um único algoritmo.